# **Prepare Data**

In [0]:
!pip install bert-embedding
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.2.3)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /

/


In [0]:
cd content/drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
cd ..

/content/drive/My Drive


In [0]:
# first copy and rename original file
# then do
import shutil
shutil.move("aclImdb.tar.gz", "/content/")


Error: ignored

In [0]:
cd /

In [0]:
cd /content

In [0]:
!tar -xf aclImdb.tar.gz

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bert_embedding import BertEmbedding
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.stem import WordNetLemmatizer
import re
from gensim.models import Word2Vec
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import html
import re
import progressbar
import itertools 
import os
import random
import numpy as np


![alt text](https://i.imgur.com/Mqf7upH.png)


# **Preprocessing**

![alt text](https://i.imgur.com/SrKn7zq.png)

In [0]:
# stoplist = set(stopwords.words('english'))
stoplist = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

def remove_stopwords(text):
    temp = ""
    words = text.split()
    for r in words:
        if not r in stoplist:
            temp = temp + " " + r
    return temp.strip()

# Stemming
def doStem(text):
    porter = PorterStemmer()
    result = ""
    for word in text.split():
        result = result + " " + porter.stem(word)
    return result.strip()

# Lemmatization
def doLemm(text):
    wordNet = WordNetLemmatizer()
    result = ""
    for word in text.split():
        result = result + " " + wordNet.lemmatize(word)
    return result.strip()

In [0]:
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return ' '.join([m.group(0) for m in matches])

def preprocess(text,rmvsw,p):

    contractions = {
        "ain't": "are not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because",
        "could've": "could have", "couldn't": "could not","couldn't've": "could not have", "didn't": "did not",
        "doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have", "hasn't": "has not",
        "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
        "he'll've": "he will have","he's": "he is","how'd": "how did","how'd'y": "how do you",
        "how'll": "how will","how's": "how is","i'd": "I would", "i'd've": "I would have","i'll": "I will","i'll've": "I will have",
        "i'm": "I am","i've": "I have","isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will",
        "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have",
        "mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
        "needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have",
        "shan't": "shall not","sha'n't": "shall not", "shan't've": "shall not have","she'd": "she would", "she'd've": "she would have",
        "she'll": "she will","she'll've": "she will have","she's": "she is","should've": "should have",
        "shouldn't": "should not","shouldn't've": "should not have", "so've": "so have","so's": "so is","that'd": "that had","that'd've": "that would have",
        "that's": "that is","there'd": "there had", "there'd've": "there would have", "there's": "there is","they'd": "they would",
        "they'd've": "they would have","they'll": "they will","they'll've": "they will have", "they're": "they are",
        "they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would","we'd've": "we would have",
        "we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what'll've": "what will have",
        "what're": "what are","what's": "what is", "what've": "what have", "when's": "when is","when've": "when have","where'd": "where did","where's": "where is","where've": "where have","who'll": "who will",
        "who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have","will've": "will have",
        "won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
        "y'all": "you all","y'all'd": "you all would","y'all'd've": "you all would have",
        "y'all're": "you all are","y'all've": "you all have","you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have","you're": "you are","you've": "you have"
    }
    if rmvsw == True:
      text = remove_stopwords(text)

    # CamleCase Split
    text = camel_case_split(text).lower() # MiladMolazadeh -> milad molazadeh

    text = text.lower()
    
    # Removing html tags
    sentence = remove_tags(text)
        
    #Apostrophe Lookup
    words = sentence.split()
    for word in sentence.split():
        if word.lower() in contractions:
            sentence = sentence.replace(word, contractions[word.lower()])
            
    sentence = html.unescape(sentence) # remove sth like &lt;3 (that defines heart)
    
    # happpppppppy -> happy
    sentence = ''.join(''.join(s)[:2] for _, s in itertools.groupby(sentence))

    # # remove urls
    sentence = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', sentence)

    
    # Single character removal
    # sentence = re.sub(r"\s+[a-zA-Z]+\s+", ' ', sentence)

    if p == False:
      # Remove punctuations and numbers
      sentence = re.sub('[^a-zA-Z\.\?\;\.\.+\!\!+]', ' ', sentence)
      sentence = re.sub('\.\.+', '.', sentence)
      sentence = re.sub('\. ', '. ', sentence)
      sentence = re.sub('\?', '. ', sentence)
      sentence = re.sub('\!', '. ', sentence)
      sentence = re.sub('\!\!+', '. ', sentence)
      sentence = re.sub('\?\?+', '. ', sentence)
      sentence = re.sub('\;\;+', '. ', sentence)
      sentence = re.sub('\:\:+', ' ', sentence)
      sentence = re.sub('[\']', ' ', sentence)
      sentence = doStem(sentence)
      sentence = doLemm(sentence)
      # Removing multiple spaces
      sentence = re.sub(r'\s+', ' ', sentence)
      return " ".join(sentence.split()).lower()
    
    else:
      sentence = re.sub('[^a-zA-Z\']', ' ', sentence)
      sentence = re.sub('\.\.+', ' ', sentence)
      sentence = re.sub('\. ', ' ', sentence)
      sentence = re.sub('\!\!+', ' ', sentence)
      sentence = re.sub('\?\?+', ' ', sentence)
      sentence = re.sub('\;\;+', ' ', sentence)
      sentence = re.sub('\:\:+', ' ', sentence)
      sentence = re.sub('[\']', ' ', sentence)
      # Single character removal
      text = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
      sentence = doStem(sentence)
      # sentence = doLemm(sentence)
      
      # Removing multiple spaces
      sentence = re.sub(r'\s+', ' ', sentence)
      return " ".join(sentence.split()).lower()



TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

sample="mr.ali how not are you? hi AwsomeBookFind"

preprocess(sample,True,True)

![alt text](https://i.imgur.com/CVC0WeL.png)

داکبومنت ها را شافل میکنیم تا مشکل پیش برازش پیش نیاید

و نتیجه ما تحت تاثیر مرتب بودن دیتا نباشد

In [0]:
%%time

def load_dataset_to_python(datapath, seed,remove_stopwords, sentence,p):
    imdb_data_path = os.path.join(datapath,"aclImdb")
    train_texts = []
    train_labels = []
    
    for category in ['pos', 'neg']:
        train_path = os.path.join(imdb_data_path, 'train', category)
        for fname in sorted(os.listdir(train_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(train_path, fname)) as f:
                        train_texts.append(preprocess(f.read(),True,p))
                        train_labels.append(0 if category == 'neg' else 1)

    test_texts = []
    test_labels = []
    
    for category in ['pos', 'neg']:
        test_path = os.path.join(imdb_data_path, 'test', category)
        for fname in sorted(os.listdir(test_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(test_path, fname)) as f:
                    test_texts.append(preprocess(f.read(),True,p))
                    test_labels.append(0 if category == 'neg' else 1)
                
    random.seed(seed)
    random.shuffle(train_texts)
    random.seed(seed)
    random.shuffle(train_labels)

    
    return ((np.array(train_texts), np.array(train_labels)), (np.array(test_texts), np.array(test_labels)))

In [0]:
%%time
train , test = load_dataset_to_python("", seed=200,remove_stopwords=False,sentence=False,p=True)

In [0]:
%%time
train_dot , test_dot = load_dataset_to_python("", seed=200,remove_stopwords=False,sentence=False,p=False)

In [0]:
%%time
train_sw , test_sw = load_dataset_to_python(
    "", seed=200,remove_stopwords=True,sentence=False,p=True)

In [0]:
%%time
train_sw_dot , test_sw_dot = load_dataset_to_python(
    "", seed=200,remove_stopwords=True,sentence=False,p=False)

# **Vectorize**

![alt text](https://i.imgur.com/MvcU0pZ.png)


In [0]:
def bag_of_words_vectorize(lstTrain,lstTest):
    cv = CountVectorizer()
    cv.fit(lstTrain)
    X = cv.transform(lstTrain)
    X_test = cv.transform(lstTest)
    
    print(r"Done! number of features are: ",len(cv.get_feature_names()))
    print(r"type of outputs are: ",type(X))

    return X,X_test


def tfidf_vectorize(lstTrain,lstTest,ngram=(1,3)):
    cv = TfidfVectorizer(binary=False, ngram_range=ngram)
    cv.fit(lstTrain)
    X = cv.transform(lstTrain)
    X_test = cv.transform(lstTest)
    
    print(r"Done! number of features are: ",len(cv.get_feature_names()))
    print(r"type of outputs are: ",type(X))
    
    return X,X_test

def word2vec(lstSentences):
    vectors = []
    for sentence in lstSentences:
      tokens = [nltk.word_tokenize(sentence)]
      word_vec = Word2Vec(tokens, min_count=1)
      vocabulary = word_vec.wv.vocab
      i=0
      for word in vocabulary:
        if i == 0:
          i=1
          v1 = word_vec.wv[word]
        else:
          v1 = sum_two_matrix(v1,word_vec.wv[word])
      vectors.append([x / len(vocabulary) for x in v1])
    return vectors

def bert_word_embedding(lstParagraphs):
    vectors = []
    v1 = []

    bert_embedding = BertEmbedding()
    return bert_embedding(lstParagraphs)
    for par in lstParagraphs:
      lstSent = par.split('. ')
      i=0
      for sent in lstSent:
        if i == 0:
          j = 0
          for j in len(v0):
            if j == 0:
              v0 = bert_embedding(sent)[i][1][j]
              j=1
            else:
              v0 = sum_two_matrix(v0,bert_embedding(sent)[i][1][j])
              j = j+1
          v1.append([x / len(v0) for x in v0])
          i = i + 1
        else:
          j = 0
          for j in len(v0):
            if j == 0:
              v0 = bert_embedding(sent)[i][1][j]
              j=1
            else:
              v0 = sum_two_matrix(v0,bert_embedding(sent)[i][1][j])
              j = j+1
          v1.append([x / len(v0) for x in v0])
          i = i + 1
      vectors.append([x / len(lstSent) for x in v1])
    return vectors

def sentence_word_embedding(lstTrain, lstTest):
    
    model = SentenceTransformer('bert-base-nli-mean-tokens')    
    return model.encode(lstTrain),model.encode(lstTest)

def sum_two_matrix(X,Y):
  return [X[i] + Y[i]  for i in range(len(X))]

# result = bert_word_embedding(['my name is milad', 'i live in iran'])

# **Classifiers**

![alt text](https://i.imgur.com/2YxiRM1.png)


In [0]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
import matplotlib.pyplot as plt

def plot_roc_curve(fpr, tpr, label=None):
  plt.plot(fpr,tpr, linewidth=3, label=label)
  plt.plot([0, 1], [0, 1], 'k--')
  plt.axis([0, 1, 0, 1])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')

def modelEvaluation(predictions, y_test_set):
    #Print model evaluation to predicted result 
    
    print ("\nAccuracy on test set: {:.4f}".format(accuracy_score(y_test_set, predictions)))
    print("---------------------------")
    print ("\nAUC score : {:.4f}".format(roc_auc_score(y_test_set, predictions)))
    print("---------------------------")
    print ("\nClassification report : \n", metrics.classification_report(y_test_set, predictions))
    print("---------------------------")
    print ("\nConfusion Matrix : \n", metrics.confusion_matrix(y_test_set, predictions))
    print("\n---------------------------\n")
    fpr, tpr, tresholds = roc_curve(predictions, y_test_set)
    plot_roc_curve(fpr,tpr)
    plt.show()

    
def naive_bayes_classifier(train, test, train_target, test_target):
    mnb = MultinomialNB()
    mnb.fit(train, train_target)
    predictions = mnb.predict(test)
    modelEvaluation(predictions, test_target)
    
    return None

def naive_bayes_classifier_gausianNB(train, test, train_target, test_target):
    gnb = GaussianNB()
    gnb.fit(train, train_target)
    predictions = gnb.predict(test)
    modelEvaluation(predictions, test_target)
    
    return None

def svm_classifier(train, test, train_target, test_target):
    X_train, X_val, y_train, y_val = train_test_split(train, train_target, train_size=0.75) 
    final = LinearSVC(C=0.5)
    final.fit(train, train_target)
    modelEvaluation(final.predict(test), test_target)
                                    
    return None


def random_forest_classifier(train, test, train_target, test_target):
    rand = RandomForestClassifier() # n_estimators is 10 
    rand.fit(train, train_target)
    pred = rand.predict(test)
    modelEvaluation(pred, test_target)
    return None


def dt_classifier(train,test, train_target,test_target):
    dt = DecisionTreeClassifier()
    dt.fit(train,train_target)
    pred = dt.predict(test)
    modelEvaluation(pred, test_target)
    return None


# **Bag of Words**

In [0]:
%%time
X, X_test = bag_of_words_vectorize(train[0],test[0])

In [0]:
%%time
svm_classifier(X, X_test, train[1], test[1])
# 0.8472 => entire document
# 0.8486 => sentence by sentence


In [0]:
naive_bayes_classifier(X, X_test, train[1], test[1])

In [0]:
%%time
random_forest_classifier(X,X_test,train[1],test[1])

In [0]:
%%time
dt_classifier(X,X_test,train[1],test[1])
# 0.7118 ver2 entire
# 0.8152 => sentence by sentence

# **Word2Vec**

In [0]:
par = "hi may name is milad. i live iniran."
sentences = par.split('. ')
print(sentences)

In [0]:
# word2vec sentence by sentence
%%time
train_vectors_snt = []

for par in train[0]:
  i=0
  sentences = par.split('. ')
  if i == 0:
    i=1
    sent_vector = word2vec(filter(None, sentences))
  else:
    sent_vector = sum_two_matrix(sent_vector,word2vec(sentences))
  train_vectors_snt.append(sent_vector)

In [0]:
print(len(train_vectors_snt))

In [0]:
# word2vec sentence by sentence
%%time
test_vectors_snt = []

for par in test[0]:
  i=0
  sentences = par.split('. ')
  if i == 0:
    i=1
    sent_vector = word2vec(filter(None, sentences))
  else:
    np.sum(sent_vector,word2vec(sentences))
  test_vectors_snt.append(sent_vector)

In [0]:
%%time
train_sw_dot_vectors = word2vec(train_sw_dot[0])

In [0]:
%%time
test_sw_dot_vectors = word2vec(test_sw_dot[0])

In [0]:
%%time
train_vectors = word2vec(train_dot[0])

In [0]:
%%time
test_vectors = word2vec(test_dot[0])

In [0]:
%%time
svm_classifier(train_vectors, test_vectors,train[1], test[1])

In [0]:
%%time
svm_classifier(train_sw_dot_vectors, test_sw_dot_vectors,train_sw_dot[1], test_sw_dot[1])

In [0]:
# naive bayes with word2vec paragraph by paragraph

%%time
naive_bayes_classifier_gausianNB(np.asarray(train_vectors), np.asarray(test_vectors),train[1], test[1])

In [0]:
%%time
naive_bayes_classifier_gausianNB(train_sw_dot_vectors, test_sw_dot_vectors,train_sw_dot[1], test_sw_dot[1])

In [0]:
# random forest with word2vec paragraph by paragraph

%%time
random_forest_classifier(train_vectors, test_vectors,train[1], test[1])

In [0]:
%%time
random_forest_classifier(train_sw_dot_vectors, test_sw_dot_vectors,train_sw_dot[1], test_sw_dot[1])

In [0]:
# decision tree with  word2vec paragraph by paragraph

%%time
dt_classifier(train_vectors, test_vectors,train[1], test[1])


In [0]:
%%time
dt_classifier(train_sw_dot_vectors, test_sw_dot_vectors,train_sw_dot[1], test_sw_dot[1])

# **BERT Sentence embedding**

In [0]:
%%time
X_wbert, X_wbert_test = sentence_word_embedding(train[0],test[0])


In [0]:
%%time
svm_classifier(X_wbert, X_wbert_test, train[1], test[1])

In [0]:
%%time
naive_bayes_classifier_gausianNB(X_wbert, X_wbert_test, train[1], test[1])

In [0]:
%%time
random_forest_classifier(X_wbert, X_wbert_test, train[1], test[1])

In [0]:
%%time
dt_classifier(X_wbert, X_wbert_test, train[1], test[1])

# **TF-IDF**


In [0]:
%%time
train_tfidf,test_tfidf = tfidf_vectorize(train[0],test[0])

In [0]:
%%time
train_tfidf_sw,test_tfidf_sw = tfidf_vectorize(
    train_sw[0],test_sw[0])

In [0]:
svm_classifier(train_tfidf, test_tfidf,train[1], test[1])

In [0]:
svm_classifier(train_tfidf_sw, 
               test_tfidf_sw,
               train_sw[1], 
               test_sw[1])

In [0]:
naive_bayes_classifier(train_tfidf, test_tfidf,train[1], test[1])

In [0]:
naive_bayes_classifier(train_tfidf_sw, 
               test_tfidf_sw,
               train_sw[1], 
               test_sw[1])

In [0]:
random_forest_classifier(train_tfidf, test_tfidf,train[1], test[1])

In [0]:
random_forest_classifier(train_tfidf_sw, 
               test_tfidf_sw,
               train_sw[1], 
               test_sw[1])

In [0]:
dt_classifier(train_tfidf, test_tfidf,train[1], test[1])

In [0]:
%%time
dt_classifier(train_tfidf_sw, 
               test_tfidf_sw,
               train_sw[1], 
               test_sw[1])

# Bert Word Embedding


In [0]:
X_wordBert = bert_word_embedding(train[0])

In [0]:
X_wordBert_test = bert_word_embedding(test[0])

In [0]:
naive_bayes_classifier(X_wbert, X_wbert_test,train[1], test[1])

In [0]:
svm_classifier(X_wbert, X_wbert_test,train[1], test[1])

In [0]:
random_forest_classifier(X_wbert, X_wbert_test,train[1], test[1])

In [0]:
dt_classifier(X_wbert, X_wbert_test,train[1], test[1])